In [8]:
from jikanpy import Jikan
import pandas as pd
import time
import os 


jikan = Jikan()

cb_bp = jikan.anime(1)

# if 'data' in cb_bp:
#     print(cb_bp['data'].keys())
# print(f"Title: {cb_bp['title']}, Source: {cb_bp['source']}")